# Exercise 04

Generation of the exercise parameters:

In [1]:
size = 20
# p = random_prime(2**size)
# q = random_prime(2**size)
p,q = 797813, 892531
N = p*q
print((N))
phi = (p-1)*(q-1)
d = 31
e = inverse_mod(d,phi)
print(e,d,(N)**(0.25)/3)

712072834703
574250922871 31 306.203287954283


In [2]:
# check of the encryption and decryption works
f = continued_fraction(e/N)
print(f'fraction expansion of e/N: {f[:7]}...')
print(f'the convergets are {f.convergents()[:4]}...')
convergents = f.convergents()
for i in range(1, 1000):
    r = convergents[i]
    d = r.denominator()
    if pow(pow(2, e, N), d, N) == 2:
        print(f'found d: {d}')
        break
        
assert d < 1/3 * N**0.25

fraction expansion of e/N: [0; 1, 4, 6, 543, 3]...
the convergets are [0, 1, 4/5, 25/31]...
found d: 31


## Solutions

### Exercises 0 and 1

In [3]:
_.<z> = GF(5)[]
f = z**3 + z**2 + 2*z + 3
print ( f.factor() )
print ( z**20 % f)

(z + 2)^3
1


In [4]:
def f(s):
    return s[-1]*4 + s[-2]*3 + s[-3]*2

d = [0, 0, GF(5)(1)]
for i in range(3, 23):
    d.append(f(d))
print(d)

[0, 0, 1, 4, 4, 0, 0, 3, 2, 2, 0, 0, 4, 1, 1, 0, 0, 2, 3, 3, 0, 0, 1]


In [1]:
_.<z> = GF(2)[]
f1 = z**19 + z**18 + z**17 + z**14 + 1
print(f1.is_primitive())
f2 = z**22 + z**21 + 1
print(f2.is_primitive())
f3 = z**23 + z**22 + z**21 + z**8 + 1
print(f3.is_primitive())

True
True
True


In [6]:
max_order = 2**22 - 1
print(max_order.factor())
ff = [3, 23, 89, 683]

3 * 23 * 89 * 683


In [7]:
for _f in ff:
    print(pow(z, max_order//_f, f2))
print(pow(z, max_order, f2))

z^21 + z^19 + z^18 + z^17 + z^14 + z^13 + z^12 + z^10 + z^6 + z^5 + z + 1
z^20 + z^19 + z^18 + z^9 + z^8 + z^6 + z^5 + z^4
z^21 + z^19 + z^17 + z^16 + z^13 + z^12 + z^11 + z^8 + z^7 + z^5 + z^4 + z^3
z^20 + z^19 + z^17 + z^16 + z^12 + z^5 + z^4 + z^3 + z + 1
1


In [1]:
orders = [ 2**19 - 1, 2**22 - 1, 2**23 - 1]
print([f'{_o} = {_o.factor()}' for _o in orders])
print(f'lcm of the numbers is {prod(orders)}')

['524287 = 524287', '4194303 = 3 * 23 * 89 * 683', '8388607 = 47 * 178481']
lcm of the numbers is 18446702292280803327


### Exercise 2

Weiner attack: we are given the public exponent and the public modulus.


In [6]:
## parameters
e = 574250922871
n = 712072834703
print(f'Encryption of 2 is {pow(2, e, n)}')

Encryption of 2 is 544315886326


The continued fraction expansion of a (rational) number $s$ is an expression of the form:
$$
    a_0 + \frac{1}{
        a_1 + \frac{1}{
            a_2 + \frac{1}{
                a_3 + \frac{1}{
                    a_4 + \frac{1}{
                        \dots
                    }
                }
            }
        }
    }
$$

It is clear the coefficients $a_0,a_1,...$ can computed recursively performing division with reminder.

In [4]:
def continued_fraction_exp(numerator, denominator):
    """
    Compute the continued fraction expansion of a rational number
    and its convergents.
    """
    cf = []
    while denominator != 0:
        quotient = numerator // denominator
        remainder = numerator % denominator
        cf.append(quotient)
        numerator, denominator = denominator, remainder
    return cf

# Example usage:
numerator = e
denominator = n
cf = continued_fraction_exp(numerator, denominator)
print(f"Continued fraction expansion: {cf}")

Continued fraction expansion: [0, 1, 4, 6, 543, 2, 1, 1, 49, 1, 3, 12, 1, 1, 4, 1, 5, 3, 1, 3, 3]


Another important properties of the continued fractions is that the $k$-th convergent can be recursively computed from the previous two convergents and the coefficients.
In particular, let $d_k$ be the $k$-th denominator, then:
$$
    d_k = a_m d_{k-1} + d_{k-2} \ .
$$
Via fixing $d_0 = 1$ and $d_1 = a_1$ we can the evaluate them all.

In [5]:
def check_rsa(d, e = e, n = n):
    return pow(pow(2, e, n), d, n) == 2

cf = continued_fraction_exp(e, n)
def find_d(cf, e = e, n = n):
    """
    Find the private exponent d from the continued fraction expansion
    of e/n.
    """
    i = 0
    numerator = e
    denominator = n

    while True:
        quotient = numerator // denominator
        remainder = numerator % denominator
        numerator, denominator = denominator, remainder
        print(f"Step {i}: quotient = {quotient}")
        if i == 0: 
            _d0 = 1
            print(f"Step {i}: d0 = {_d0}")
            if check_rsa(_d0):
                return _d0
        elif i == 1:
            _d1 = quotient
            print(f"Step {i}: d1 = {_d1}, d0 = {_d0}")
            if check_rsa(_d1):
                return _d1
        else:
            d = quotient * _d1 +  _d0
            print(f"Step {i}: d = {d} = {quotient} * {_d1} + {_d0}")
            if check_rsa(d):
                return d
            _d0, _d1 = _d1, d
        i += 1
        
find_d(cf)

Step 0: quotient = 0
Step 0: d0 = 1
Step 1: quotient = 1
Step 1: d1 = 1, d0 = 1
Step 2: quotient = 4
Step 2: d = 5 = 4 * 1 + 1
Step 3: quotient = 6
Step 3: d = 31 = 6 * 5 + 1


31

In [13]:
from sympy import randprime, nextprime, isprime

while True:
    p = randprime(2**127, 2**128)
    q = nextprime(p)
    if isprime((p-1) // 2) and isprime((q-1) // 2):
        break
n = p * q
print(hex(n))

0x4b1d6ec23470ee5bbce81aca3286cc42d0967b865eb54529f7a0ea2a64661711


In [ ]:
n = 0xc3348eace373f1f97fe6f64f4145b3bcf2488f31170c0ec8a404466029d55ded 
n.factor(verbose = 4)


In [7]:
n = 0xc3348eace373f1f97fe6f64f4145b3bcf2488f31170c0ec8a404466029d55ded 

In [8]:
def fermat_factorization(n):
    if n <= 0:
        raise ValueError("The number must be a positive integer.")

    # Check if n is a perfect square
    if n.is_square():
        return (n.isqrt(), n.isqrt())

    x = n.isqrt()
    if x * x == n:
        return (x, x)

    x += 1 if x * x < n else 0

    while True:
        y2 = x * x - n
        if y2.is_square():
            y = y2.isqrt()
            return (x + y, x - y)
        x += 1

In [9]:
p,q = fermat_factorization(n)
print(f'p: {p}, q: {q}, |p-q| = {abs(p-q)}')

p: 297142838093316797395239584655506734007, q: 297142838093316797395239584655506733947, |p-q| = 60


In [21]:
p = next_prime(n.sqrt().round())
n % p

0

### Exercise 3

In [10]:
(6*14 + 20*9 + 3*4) % 29

15

In [34]:
p = 29
F = GF(p)  # Define the finite field GF(p)
R.<x> = PolynomialRing(F)  # Define polynomial ring over GF(p)

def berlekamp_massey(s):
    n = len(s)  # Length of sequence
    L, m, b = 0, 1, F(1)  # LFSR parameters
    C = R(1)  # Connection polynomial (1 initially)
    B = R(1)  # Temporary polynomial
    
    print(f"Initial setup: L={L}, m={m}, b={b}, C={C}")
    
    print(f'Sequences: {s}')
    for N in range(n):
        # first we check if the LFSR equation is verified
        d = sum (C.list()[i] * s[N - i] for i in range(0,L+ 1) if i <= N and i <= C.degree() )  # Discrepancy
        d = F(d)
        print(f"Step {N}: Discrepancy d = {d}")
        
        if d == 0:
            m += 1  # No update needed, just increment m
        else:
            T = C  # Copy of C
            coeff = d / b  # Correction factor
            C -= coeff * x^m * B  # Update C using polynomial operations
            
            print(f"Updated C: {C}")
            
            if 2 * L <= N:
                L = N + 1 - L
                B = T  # Save old C
                b = d  # Update b
                m = 1  # Reset m
                print(f"L updated to {L}, new B={B}")
            else:
                m += 1
    
    print(f"Final connection polynomial: {C}")
    return C, L

s = [4, 9, 14, 22, 13, 6]  # Example sequence
chi,L = berlekamp_massey(s)


Initial setup: L=0, m=1, b=1, C=1
Sequences: [4, 9, 14, 22, 13, 6]
Step 0: Discrepancy d = 4
Updated C: 25*x + 1
L updated to 1, new B=1
Step 1: Discrepancy d = 22
Updated C: 5*x + 1
Step 2: Discrepancy d = 1
Updated C: 7*x^2 + 5*x + 1
L updated to 2, new B=5*x + 1
Step 3: Discrepancy d = 10
Updated C: 15*x^2 + 24*x + 1
Step 4: Discrepancy d = 26
Updated C: 15*x^3 + 18*x^2 + 24*x + 1
L updated to 3, new B=15*x^2 + 24*x + 1
Step 5: Discrepancy d = 25
Updated C: 24*x^3 + 15*x^2 + 13*x + 1
Final connection polynomial: 24*x^3 + 15*x^2 + 13*x + 1


In [27]:
x**L * chi(1/x)

x^3 + 13*x^2 + 15*x + 24